# SuperAI Season 4 - Level 2 Hackathon - Forest Type Double Classifier

## Explore Data

In [1]:
import pandas as pd
import numpy as np

In [2]:
train_df = pd.read_csv('./datasets/train.csv' , index_col='id')
test_df = pd.read_csv('./datasets/test.csv' , index_col='id')

## Create NON-DEF (MDF / DDF) / DEF 

In [3]:
DEF_df = train_df[train_df['nforest_type'] == 'DEF']
NON_DEF_df = train_df[train_df['nforest_type'] != 'DEF']

In [4]:
NON_DEF_df.to_csv('./datasets/MDF_DDF.csv')

In [5]:
NON_DEF_df['nforest_type'] = 'NON_DEF'
DEF_df['nforest_type'] = 'DEF'

C:\Users\teehe\AppData\Local\Temp\ipykernel_20136\3381345259.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  NON_DEF_df['nforest_type'] = 'NON_DEF'
C:\Users\teehe\AppData\Local\Temp\ipykernel_20136\3381345259.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DEF_df['nforest_type'] = 'DEF'


In [6]:
NON_DEF_df

,b1,b11,b12,b2,b3,b4,b5,b6,b7,b8,b8_a,b9,nforest_type
id,,,,,,,,,,,,,
2002,293,1927,1038,278,475,453,987,1773,2184,1900,2343,3039,NON_DEF
3212,197,1598,697,201,347,228,682,1982,2449,2254,2685,2690,NON_DEF
13312,929,1975,1031,982,1020,856,1220,2051,2421,2392,2671,2683,NON_DEF
17020,132,1560,689,189,408,175,609,2117,2907,3024,3005,2955,NON_DEF
5967,241,1944,1131,362,538,487,918,1549,1844,1702,2077,2043,NON_DEF
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9185,374,1940,1054,382,565,498,977,1678,1929,2109,2291,2100,NON_DEF
13977,1983,3602,2720,1622,1782,1766,2314,3488,3900,3924,4097,6053,NON_DEF
755,940,2007,1148,975,1080,968,1252,1780,1983,1942,2247,2170,NON_DEF


In [7]:
DEF_df

,b1,b11,b12,b2,b3,b4,b5,b6,b7,b8,b8_a,b9,nforest_type
id,,,,,,,,,,,,,
14932,701,1776,895,689,905,684,1187,2504,2836,2839,3172,3103,DEF
14572,156,1121,547,264,472,362,699,1673,1967,2182,2260,2277,DEF
7504,536,1591,777,686,760,624,1049,2152,2600,2665,2964,2777,DEF
10489,1551,1519,718,1730,1764,1514,1719,2267,2554,2598,2743,2743,DEF
7995,313,1696,864,366,568,411,849,1965,2387,2439,2827,2759,DEF
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3762,182,1336,574,188,296,145,571,2000,2473,2243,2884,2844,DEF
7804,1,1753,820,106,431,301,840,2091,2472,2762,2914,2440,DEF
5566,281,1837,835,342,718,285,975,2801,3556,3637,3801,3148,DEF


In [8]:
DEF_NONDEF_df = pd.concat([NON_DEF_df , DEF_df])
DEF_NONDEF_df.sort_index().to_csv('./datasets/DEF_NONDEF.csv')

## DEF_NONDEF Classifier

In [9]:
from autogluon.tabular import TabularPredictor

In [10]:
DEF_NONDEF_df = pd.read_csv('./datasets/DEF_NONDEF.csv' , index_col='id')
test_df = pd.read_csv('./datasets/test.csv', index_col = 'id')

In [11]:
def add_features_for_DEF_NONDEF(row) :

    row['NDVI'] = (row['b8'] - row['b4']) / (row['b8'] + row['b4'])
    row['EVI'] = 2.5 * ((row['b8'] - row['b4']) / (row['b8'] + 6 * row['b4'] - 7.5 * row['b2'] + 1.01))
    row['NDWI '] = (row['b3'] - row['b8']) / (row['b3'] + row['b8'])
    row['SAVI '] = (row['b8'] - row['b4']) * (1 + 0.5) / (row['b8'] + row['b4'] + 0.5)
    row['MSAVI'] = (2 * row['b8'] + 1 - ( (2 * row['b8'] + 1) ** 2 - 8 * (row['b8'] - row['b4'])) ** (1 / 2)) / 2
    row['GNDVI '] = (row['b8'] - row['b3']) / (row['b8'] + row['b3'])
    row['RENDVI '] = (row['b8'] - row['b5']) / (row['b8'] + row['b5'])
    row['NDMI '] = (row['b8'] - row['b11']) / (row['b8'] + row['b11'])
    row['GRVI'] = (row['b3'] - row['b4']) / (row['b3'] + row['b4'])
    row['TVI'] = ( (row['b8'] - row['b4']) / (row['b8'] + row['b4'] + 0.5) ) ** (1 / 2)
    row['MCARI'] = ((row['b5'] - row['b4']) - 0.2 * (row['b5'] - row['b3'])) / (row['b5'] / row['b4'])
    row['BSI'] =  ((row['b11'] + row['b4']) - (row['b8'] + row['b2'])) / ((row['b11'] + row['b4']) + (row['b8'] + row['b2']))
    row['NBR'] = (row['b8'] - row['b12']) / (row['b8'] + row['b12'])
    row['MSI'] = row['b11'] / row['b8']

    return row

def drop_features_for_DEF_NONDEF(df) :

    return df
    # return df.drop(columns = ['b1'])


In [12]:
DEF_NONDEF_df = DEF_NONDEF_df.apply(add_features_for_DEF_NONDEF , axis = 1)
test_df = test_df.apply(add_features_for_DEF_NONDEF , axis = 1)

DEF_NONDEF_df = drop_features_for_DEF_NONDEF(DEF_NONDEF_df)
test_df = drop_features_for_DEF_NONDEF(test_df)

In [13]:
DEF_NONDEF_df['nforest_type'].value_counts()

nforest_type
NON_DEF    10468
DEF         2585
Name: count, dtype: int64

In [14]:
from imblearn.over_sampling import SMOTE , KMeansSMOTE
from imblearn.combine import SMOTEENN , SMOTETomek

smote = SMOTE(random_state = 42 , sampling_strategy= 'all')

DEF_NONDEF_df , label_df  = smote.fit_resample(DEF_NONDEF_df.drop(columns=['nforest_type']) , DEF_NONDEF_df['nforest_type'])
DEF_NONDEF_df = DEF_NONDEF_df.join(label_df)

c:\anaconda\Lib\site-packages\joblib\externals\loky\backend\context.py:110: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] The system cannot find the file specified
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "c:\anaconda\Lib\site-packages\joblib\externals\loky\backend\context.py", line 199, in _count_physical_cores
    cpu_info = subprocess.run(
               ^^^^^^^^^^^^^^^
  File "c:\anaconda\Lib\subprocess.py", line 548, in run
    with Popen(*popenargs, **kwargs) as process:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\anaconda\Lib\subprocess.py", line 1026, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "c:\anaconda\Lib\subprocess.py", line 1538, in _execute_child
    hp, ht, pid, tid = _winapi.CreateProcess(executable, args,
                       ^^^^^^^^^^^^^^^^^^^

In [15]:
DEF_NONDEF_df['nforest_type'].value_counts()

nforest_type
NON_DEF    10468
DEF        10468
Name: count, dtype: int64

In [ ]:
hyperparameters = {
    'GBM': {
        'num_boost_round': Int(50, 1000),
        'learning_rate': Real(1e-4, 1e-1),
        'num_leaves': Int(20, 200)
    },
    'CAT': {
        'iterations': Int(100, 1000),
        'depth': Int(4, 10),
        'learning_rate': Real(1e-4, 1e-1)
    },
    'NN_TORCH': {
        'epochs': Int(5, 100),
        'lr': Real(1e-4, 1e-2),
        'dropout_prob': Real(0.0, 0.5)
    },
    'XGB': {
        'n_estimators': Int(50, 1000),
        'learning_rate': Real(1e-4, 1e-1),
        'max_depth': Int(3, 10)
    }
}


In [16]:
label = 'nforest_type'
predictor = TabularPredictor(label = label).fit(
    DEF_NONDEF_df,
    time_limit=3600,  # 1 hour
    hyperparameters=hyperparameters,
    search_strategy='bayes',
    num_trials=100  # Number of hyperparameter configurations to try)
)

No path specified. Models will be saved in: "AutogluonModels\ag-20240604_185835"
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels\ag-20240604_185835"
=================== System Info ===================
AutoGluon Version:  1.1.0
Python Version:     3.11.7
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22631
CPU Count:          16
Memory Avail:      

[1000]	valid_set's binary_error: 0.0864374
[2000]	valid_set's binary_error: 0.0687679
[3000]	valid_set's binary_error: 0.0592168
[4000]	valid_set's binary_error: 0.0573066
[5000]	valid_set's binary_error: 0.056829
[6000]	valid_set's binary_error: 0.0563515


	0.9465	 = Validation score   (accuracy)
	11.43s	 = Training   runtime
	0.3s	 = Validation runtime
Fitting model: LightGBM ...


[1000]	valid_set's binary_error: 0.0673352


	0.9389	 = Validation score   (accuracy)
	3.2s	 = Training   runtime
	0.05s	 = Validation runtime
Fitting model: RandomForestGini ...
	0.9207	 = Validation score   (accuracy)
	4.46s	 = Training   runtime
	0.12s	 = Validation runtime
Fitting model: RandomForestEntr ...
	0.9226	 = Validation score   (accuracy)
	5.55s	 = Training   runtime
	0.12s	 = Validation runtime
Fitting model: CatBoost ...
	0.9451	 = Validation score   (accuracy)
	73.69s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: ExtraTreesGini ...
	0.9236	 = Validation score   (accuracy)
	1.82s	 = Training   runtime
	0.12s	 = Validation runtime
Fitting model: ExtraTreesEntr ...
	0.9193	 = Validation score   (accuracy)
	1.53s	 = Training   runtime
	0.11s	 = Validation runtime
Fitting model: NeuralNetFastAI ...
	0.9136	 = Validation score   (accuracy)
	30.76s	 = Training   runtime
	0.04s	 = Validation runtime
Fitting model: XGBoost ...
	0.9394	 = Validation score   (accuracy)
	4.56s	 = Training   runtime
	0.03s

[1000]	valid_set's binary_error: 0.0582617


	0.9446	 = Validation score   (accuracy)
	8.31s	 = Training   runtime
	0.08s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	Ensemble Weights: {'NeuralNetTorch': 0.333, 'KNeighborsDist': 0.133, 'RandomForestGini': 0.133, 'CatBoost': 0.133, 'ExtraTreesGini': 0.133, 'LightGBMXT': 0.067, 'RandomForestEntr': 0.067}
	0.9599	 = Validation score   (accuracy)
	0.19s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 289.9s ... Best model: "WeightedEnsemble_L2"
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("AutogluonModels\ag-20240604_185835")


In [17]:
predictor.fit_pseudolabel(test_df)

Given test_data for pseudo labeling did not contain labels. AutoGluon will assign pseudo labels to data and use it for extra training data...
Beginning iteration 1 of pseudolabeling out of max 3
Pseudolabeling algorithm confidently assigned pseudolabels to 428 rows of data on iteration 1. Adding to train data
Fitting 13 L1 models ...
Fitting model: KNeighborsUnif_PSEUDO_1 ...
	0.8988	 = Validation score   (accuracy)
	0.04s	 = Training   runtime
	0.16s	 = Validation runtime
Fitting model: KNeighborsDist_PSEUDO_1 ...
	0.9083	 = Validation score   (accuracy)
	0.04s	 = Training   runtime
	0.14s	 = Validation runtime
Fitting model: LightGBMXT_PSEUDO_1 ...


[1000]	valid_set's binary_error: 0.0816619
[2000]	valid_set's binary_error: 0.0635148
[3000]	valid_set's binary_error: 0.0577841
[4000]	valid_set's binary_error: 0.0558739
[5000]	valid_set's binary_error: 0.0553964
[6000]	valid_set's binary_error: 0.0534862
[7000]	valid_set's binary_error: 0.0544413


	0.9475	 = Validation score   (accuracy)
	10.74s	 = Training   runtime
	0.21s	 = Validation runtime
Fitting model: LightGBM_PSEUDO_1 ...


[1000]	valid_set's binary_error: 0.0639924
[2000]	valid_set's binary_error: 0.0553964


	0.947	 = Validation score   (accuracy)
	4.3s	 = Training   runtime
	0.07s	 = Validation runtime
Fitting model: RandomForestGini_PSEUDO_1 ...
	0.9217	 = Validation score   (accuracy)
	3.48s	 = Training   runtime
	0.11s	 = Validation runtime
Fitting model: RandomForestEntr_PSEUDO_1 ...
	0.9222	 = Validation score   (accuracy)
	4.86s	 = Training   runtime
	0.1s	 = Validation runtime
Fitting model: CatBoost_PSEUDO_1 ...
	0.9446	 = Validation score   (accuracy)
	93.13s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: ExtraTreesGini_PSEUDO_1 ...
	0.9174	 = Validation score   (accuracy)
	1.43s	 = Training   runtime
	0.11s	 = Validation runtime
Fitting model: ExtraTreesEntr_PSEUDO_1 ...
	0.9198	 = Validation score   (accuracy)
	1.41s	 = Training   runtime
	0.11s	 = Validation runtime
Fitting model: NeuralNetFastAI_PSEUDO_1 ...
	0.9174	 = Validation score   (accuracy)
	24.05s	 = Training   runtime
	0.04s	 = Validation runtime
Fitting model: XGBoost_PSEUDO_1 ...
	0.9389	 = Vali

...

	0.9494	 = Validation score   (accuracy)
	13.98s	 = Training   runtime
	0.51s	 = Validation runtime
Fitting model: LightGBM_PSEUDO_2 ...


[1000]	valid_set's binary_error: 0.0711557


	0.9355	 = Validation score   (accuracy)
	2.91s	 = Training   runtime
	0.04s	 = Validation runtime
Fitting model: RandomForestGini_PSEUDO_2 ...
	0.9212	 = Validation score   (accuracy)
	3.39s	 = Training   runtime
	0.11s	 = Validation runtime
Fitting model: RandomForestEntr_PSEUDO_2 ...
	0.9202	 = Validation score   (accuracy)
	4.99s	 = Training   runtime
	0.12s	 = Validation runtime
Fitting model: CatBoost_PSEUDO_2 ...
	0.9432	 = Validation score   (accuracy)
	55.62s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: ExtraTreesGini_PSEUDO_2 ...
	0.9183	 = Validation score   (accuracy)
	1.57s	 = Training   runtime
	0.11s	 = Validation runtime
Fitting model: ExtraTreesEntr_PSEUDO_2 ...
	0.9179	 = Validation score   (accuracy)
	1.56s	 = Training   runtime
	0.12s	 = Validation runtime
Fitting model: NeuralNetFastAI_PSEUDO_2 ...
	0.914	 = Validation score   (accuracy)
	26.88s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: XGBoost_PSEUDO_2 ...
	0.9374	 = Va

[1000]	valid_set's binary_error: 0.0573066


	0.9479	 = Validation score   (accuracy)
	9.86s	 = Training   runtime
	0.1s	 = Validation runtime
Fitting model: WeightedEnsemble_L2_PSEUDO_2 ...
	Ensemble Weights: {'KNeighborsDist_PSEUDO_2': 0.2, 'NeuralNetTorch_PSEUDO_2': 0.2, 'LightGBMXT_PSEUDO_2': 0.133, 'ExtraTreesGini_PSEUDO_2': 0.133, 'NeuralNetFastAI_PSEUDO_2': 0.133, 'KNeighborsUnif_PSEUDO_2': 0.067, 'CatBoost_PSEUDO_2': 0.067, 'ExtraTreesEntr_PSEUDO_2': 0.067}
	0.9575	 = Validation score   (accuracy)
	0.16s	 = Training   runtime
	0.0s	 = Validation runtime
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("AutogluonModels\ag-20240604_185835")
Pseudolabeling algorithm changed validation score from: 0.9617956064947469, to: 0.9617956064947469 using evaluation metric: accuracy


In [18]:
prediction = predictor.predict(test_df)

In [19]:
feature_importances = predictor.feature_importance(DEF_NONDEF_df)
feature_importances

Computing feature importance via permutation shuffling for 26 features using 5000 rows with 5 shuffle sets...


	201.58s	= Expected runtime (40.32s per shuffle set)
	84.8s	= Actual runtime (Completed 5 of 5 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
b11,0.25460,0.004688,1.379135e-08,5,0.264253,0.244947
b8_a,0.19660,0.004860,4.477681e-08,5,0.206607,0.186593
b9,0.13752,0.004134,9.789948e-08,5,0.146032,0.129008
b6,0.11300,0.002864,4.944512e-08,5,0.118896,0.107104
b8,0.10600,0.005501,8.672369e-07,5,0.117326,0.094674
b7,0.10260,0.004002,2.773551e-07,5,0.110841,0.094359
b5,0.09352,0.001879,1.956009e-08,5,0.097390,0.089650
b1,0.08832,0.005330,1.584318e-06,5,0.099295,0.077345
b12,0.08412,0.006008,3.100690e-06,5,0.096490,0.071750
NBR,0.06740,0.004808,3.087280e-06,5,0.077300,0.057500


In [20]:
feature_importances.to_csv('./features/DEF_NONDEF_features_importance.csv')

In [21]:
prediction.value_counts()

nforest_type
NON_DEF    3001
DEF         999
Name: count, dtype: int64

In [22]:
prediction = prediction.sort_index()

In [23]:
prediction

id
1        NON_DEF
2        NON_DEF
5        NON_DEF
7        NON_DEF
12           DEF
          ...   
17039    NON_DEF
17042    NON_DEF
17043    NON_DEF
17047    NON_DEF
17052    NON_DEF
Name: nforest_type, Length: 4000, dtype: object

In [24]:
prediction_DEF_ONLY = prediction[prediction == 'DEF']
prediction_DEF_ONLY

id
12       DEF
19       DEF
22       DEF
29       DEF
50       DEF
        ... 
17013    DEF
17024    DEF
17026    DEF
17032    DEF
17034    DEF
Name: nforest_type, Length: 999, dtype: object

In [25]:
test_df = test_df.sort_index()

## MDF_DDF Classifier

In [26]:
test_df = pd.read_csv('./datasets/test.csv', index_col='id')

In [27]:
test_df = test_df[prediction == 'NON_DEF']

C:\Users\teehe\AppData\Local\Temp\ipykernel_20136\254591184.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  test_df = test_df[prediction == 'NON_DEF']


In [28]:
MDF_DDF_df = pd.read_csv('./datasets/MDF_DDF.csv' , index_col='id')

In [29]:
def add_features_for_MDF_DDF(row) :

    row['NDVI'] = (row['b8'] - row['b4']) / (row['b8'] + row['b4'])
    row['EVI'] = 2.5 * ((row['b8'] - row['b4']) / (row['b8'] + 6 * row['b4'] - 7.5 * row['b2'] + 1.01))
    row['NDWI '] = (row['b3'] - row['b8']) / (row['b3'] + row['b8'])
    row['SAVI '] = (row['b8'] - row['b4']) * (1 + 0.5) / (row['b8'] + row['b4'] + 0.5)
    row['MSAVI'] = (2 * row['b8'] + 1 - ( (2 * row['b8'] + 1) ** 2 - 8 * (row['b8'] - row['b4'])) ** (1 / 2)) / 2
    row['GNDVI '] = (row['b8'] - row['b3']) / (row['b8'] + row['b3'])
    row['RENDVI '] = (row['b8'] - row['b5']) / (row['b8'] + row['b5'])
    row['NDMI '] = (row['b8'] - row['b11']) / (row['b8'] + row['b11'])
    row['GRVI'] = (row['b3'] - row['b4']) / (row['b3'] + row['b4'])
    row['TVI'] = ( (row['b8'] - row['b4']) / (row['b8'] + row['b4'] + 0.5) ) ** (1 / 2)
    row['MCARI'] = ((row['b5'] - row['b4']) - 0.2 * (row['b5'] - row['b3'])) / (row['b5'] / row['b4'])
    row['BSI'] =  ((row['b11'] + row['b4']) - (row['b8'] + row['b2'])) / ((row['b11'] + row['b4']) + (row['b8'] + row['b2']))
    row['NBR'] = (row['b8'] - row['b12']) / (row['b8'] + row['b12'])
    row['MSI'] = row['b11'] / row['b8']

    return row

def drop_features_for_MDF_DDF(df) :

    return df
    # return df.drop(columns = ['b1'])

In [30]:
MDF_DDF_df = MDF_DDF_df.apply(add_features_for_MDF_DDF , axis = 1)
MDF_DDF_df = drop_features_for_MDF_DDF(MDF_DDF_df)

test_df = test_df.apply(add_features_for_MDF_DDF , axis = 1)
test_df = drop_features_for_MDF_DDF(test_df)

In [31]:
MDF_DDF_df['nforest_type'].value_counts()

nforest_type
MDF    5865
DDF    4603
Name: count, dtype: int64

In [32]:
from imblearn.over_sampling import SMOTE , KMeansSMOTE
from imblearn.combine import SMOTEENN , SMOTETomek

smote = SMOTE(random_state = 42 , sampling_strategy= 'all')

MDF_DDF_df , label_df  = smote.fit_resample(MDF_DDF_df.drop(columns=['nforest_type']) , MDF_DDF_df['nforest_type'])
MDF_DDF_df = MDF_DDF_df.join(label_df)

In [33]:
MDF_DDF_df['nforest_type'].value_counts()

nforest_type
MDF    5865
DDF    5865
Name: count, dtype: int64

In [34]:
predictor_for_MDF_DDF = TabularPredictor(label = label).fit(MDF_DDF_df)

No path specified. Models will be saved in: "AutogluonModels\ag-20240604_191418"
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels\ag-20240604_191418"
=================== System Info ===================
AutoGluon Version:  1.1.0
Python Version:     3.11.7
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22631
CPU Count:          16
Memory Avail:      

[1000]	valid_set's binary_error: 0.244672


	0.7621	 = Validation score   (accuracy)
	2.1s	 = Training   runtime
	0.04s	 = Validation runtime
Fitting model: LightGBM ...


[1000]	valid_set's binary_error: 0.236999


	0.7681	 = Validation score   (accuracy)
	1.8s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: RandomForestGini ...
	0.7613	 = Validation score   (accuracy)
	2.11s	 = Training   runtime
	0.1s	 = Validation runtime
Fitting model: RandomForestEntr ...
	0.7579	 = Validation score   (accuracy)
	3.1s	 = Training   runtime
	0.1s	 = Validation runtime
Fitting model: CatBoost ...
	0.7442	 = Validation score   (accuracy)
	10.37s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: ExtraTreesGini ...
	0.7656	 = Validation score   (accuracy)
	1.13s	 = Training   runtime
	0.1s	 = Validation runtime
Fitting model: ExtraTreesEntr ...
	0.7621	 = Validation score   (accuracy)
	1.21s	 = Training   runtime
	0.1s	 = Validation runtime
Fitting model: NeuralNetFastAI ...
	0.7596	 = Validation score   (accuracy)
	15.14s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: XGBoost ...
	0.7613	 = Validation score   (accuracy)
	5.4s	 = Training   runtime
	0.03s	 = Val

In [35]:
predictor_for_MDF_DDF.fit_pseudolabel(test_df)

Given test_data for pseudo labeling did not contain labels. AutoGluon will assign pseudo labels to data and use it for extra training data...
Beginning iteration 1 of pseudolabeling out of max 3
Pseudolabeling algorithm confidently assigned pseudolabels to 86 rows of data on iteration 1. Adding to train data
Fitting 13 L1 models ...
Fitting model: KNeighborsUnif_PSEUDO_1 ...
	0.728	 = Validation score   (accuracy)
	0.02s	 = Training   runtime
	0.11s	 = Validation runtime
Fitting model: KNeighborsDist_PSEUDO_1 ...
	0.7434	 = Validation score   (accuracy)
	0.04s	 = Training   runtime
	0.11s	 = Validation runtime
Fitting model: LightGBMXT_PSEUDO_1 ...


[1000]	valid_set's binary_error: 0.232737


	0.7681	 = Validation score   (accuracy)
	1.8s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: LightGBM_PSEUDO_1 ...


[1000]	valid_set's binary_error: 0.235294


	0.7707	 = Validation score   (accuracy)
	1.47s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: RandomForestGini_PSEUDO_1 ...
	0.7647	 = Validation score   (accuracy)
	2.19s	 = Training   runtime
	0.11s	 = Validation runtime
Fitting model: RandomForestEntr_PSEUDO_1 ...
	0.7596	 = Validation score   (accuracy)
	3.28s	 = Training   runtime
	0.11s	 = Validation runtime
Fitting model: CatBoost_PSEUDO_1 ...
	0.7587	 = Validation score   (accuracy)
	15.64s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: ExtraTreesGini_PSEUDO_1 ...
	0.757	 = Validation score   (accuracy)
	1.3s	 = Training   runtime
	0.11s	 = Validation runtime
Fitting model: ExtraTreesEntr_PSEUDO_1 ...
	0.7562	 = Validation score   (accuracy)
	1.31s	 = Training   runtime
	0.11s	 = Validation runtime
Fitting model: NeuralNetFastAI_PSEUDO_1 ...
	0.7468	 = Validation score   (accuracy)
	14.95s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: XGBoost_PSEUDO_1 ...
	0.7596	 = Val

In [36]:
prediction_MDF_DDF = predictor_for_MDF_DDF.predict(add_features_for_MDF_DDF( test_df) )

In [37]:
prediction_MDF

NameError: name 'prediction_MDF' is not defined

In [ ]:
feature_importances = predictor_for_MDF_DDF.feature_importance(MDF_DDF_df)
feature_importances

Computing feature importance via permutation shuffling for 26 features using 5000 rows with 5 shuffle sets...
	119.86s	= Expected runtime (23.97s per shuffle set)
	40.46s	= Actual runtime (Completed 5 of 5 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
b9,0.28252,0.006830,4.096252e-08,5,0.296584,0.268456
b7,0.22984,0.006956,1.005597e-07,5,0.244163,0.215517
b8,0.19548,0.007180,2.180129e-07,5,0.210264,0.180696
b12,0.19240,0.004707,4.296887e-08,5,0.202093,0.182707
b11,0.18432,0.005509,9.566407e-08,5,0.195664,0.172976
b5,0.18248,0.003495,1.613175e-08,5,0.189675,0.175285
b8_a,0.18012,0.006836,2.485013e-07,5,0.194196,0.166044
b6,0.16544,0.005070,1.057334e-07,5,0.175880,0.155000
b1,0.13084,0.002286,1.118699e-08,5,0.135548,0.126132
b3,0.11756,0.001322,1.919343e-09,5,0.120282,0.114838


In [ ]:
feature_importances.to_csv('./features/MDF_DDF_features_importance.csv')

In [ ]:
prediction_MDF_DDF.value_counts()

nforest_type
DDF    1582
MDF    1419
Name: count, dtype: int64

In [ ]:
submission_df = pd.concat([prediction_DEF_ONLY , prediction_MDF_DDF])
submission_df = submission_df.sort_index()
submission_df

id
1        MDF
2        DDF
5        MDF
7        DDF
12       DEF
        ... 
17039    DDF
17042    DDF
17043    DDF
17047    MDF
17052    DDF
Name: nforest_type, Length: 4000, dtype: object

In [ ]:
submission_df.value_counts()

nforest_type
DDF    1582
MDF    1419
DEF     999
Name: count, dtype: int64

In [ ]:
submission_path = './submissions'
submission_df.to_csv(f'{submission_path}/submission_2over_features_smote_pseudolabeling_hyper_double_classifier.csv')